In [1]:
import pyspark

from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.sql.functions import *

import pandas as pd
from IPython.display import display

#DEBUGGING
pd.options.display.max_columns = None
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', None)

from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql.functions import explode

#input_file_name,col,lit,explode,instr,regexp_replace,when,from_unixtime,unix_timestamp,trim,concat,collect_list,concat_ws
from sys import exc_info

import pandas as pd
import json
import pyarrow as pa
from pyarrow import flight

In [2]:
import os
from pyspark.sql import SparkSession, SQLContext
spark_session = SparkSession(spark).builder.getOrCreate()

In [3]:
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()
asset_base_dir=wslib.mount.get_base_dir()

In [4]:
sc = spark_session.sparkContext


In [12]:
#sc.getConf().getAll()

In [13]:
!ls '/project_data/data_asset/wx_prompt_details 2.json'

'/project_data/data_asset/wx_prompt_details 2.json'


In [9]:
wslib.load_data('test_input.json')


In [6]:
#spark.conf.set("spark.sql.streaming.schemaInference", "true")
df = spark.read.json('/project_data/data_asset/test_input.json',multiLine=True)
df.show()

+--------------------+
|           resources|
+--------------------+
|[{{{b8b849e5-1772...|
+--------------------+



In [5]:
import json

In [6]:
file_list =['test_input.json']


In [7]:
json_file_content = wslib.load_data('test_input.json')
with json_file_content as f: 
    #payload = json.load(f)
    payload = json.loads(f.read())
    print(f"{payload}")

{'resources': [{'entity': {'asset': {'id': 'b8b849e5-1772-4742-880e-7425cd854838'}, 'batch': {}, 'custom': {}, 'deployed_asset_type': 'function', 'hardware_spec': {'id': 'c076e82c-b2a7-4d20-9c0f-1f0c2fdf5a24', 'name': 'M', 'num_nodes': 1}, 'name': 'Batch dep API sleep', 'space_id': '7b648ffc-53e3-4b16-8e59-3c55ef755f31', 'status': {'state': 'ready'}}, 'metadata': {'created_at': '2023-11-08T17:07:23.026Z', 'id': '8581afca-3e73-49c8-ac0b-b694ddcdec87', 'modified_at': '2023-11-08T17:07:23.026Z', 'name': 'Batch dep API sleep', 'owner': '1000330999', 'space_id': '7b648ffc-53e3-4b16-8e59-3c55ef755f31'}}]}


In [13]:
def process_file(file_list):
    #print(file_list)
    def get_file(files):
        for file in files:
            json_file_content = wslib.load_data(file)
            with json_file_content as f: 
                payload = json.loads(f.read())
                #print(f"Processing file {payload}")
            yield payload
    return get_file(file_list)

In [14]:
file_list = ['wx_prompt_details 2.json', 'wx_prompt_details 3.json']#, 'wx_prompt_details 3.json']

rdd = sc.parallelize(file_list,2)
rdd = rdd.mapPartitions(process_file)
df3 = spark.read.json(rdd, multiLine=True, encoding="utf8" )
df3.show()

+--------------------+
|     _corrupt_record|
+--------------------+
|{'metadata': {'sp...|
|{'metadata': {'sp...|
+--------------------+



In [15]:
test_json = [{'resources': [{'entity': {'asset': {'id': 'b8b849e5-1772-4742-880e-7425cd854838'}, 'batch': {}, 'custom': {}, 'deployed_asset_type': 'function', 'hardware_spec': {'id': 'c076e82c-b2a7-4d20-9c0f-1f0c2fdf5a24', 'name': 'M', 'num_nodes': 1}, 'name': 'Batch dep API sleep', 'space_id': '7b648ffc-53e3-4b16-8e59-3c55ef755f31', 'status': {'state': 'ready'}}, 'metadata': {'created_at': '2023-11-08T17:07:23.026Z', 'id': '8581afca-3e73-49c8-ac0b-b694ddcdec87', 'modified_at': '2023-11-08T17:07:23.026Z', 'name': 'Batch dep API sleep', 'owner': '1000330999', 'space_id': '7b648ffc-53e3-4b16-8e59-3c55ef755f31'}}]},{'resources': [{'entity': {'asset': {'id': 'b8b849e5-1772-4742-880e-7425cd854838'}, 'batch': {}, 'custom': {}, 'deployed_asset_type': 'function', 'hardware_spec': {'id': 'c076e82c-b2a7-4d20-9c0f-1f0c2fdf5a24', 'name': 'M', 'num_nodes': 1}, 'name': 'Batch dep API sleep', 'space_id': '7b648ffc-53e3-4b16-8e59-3c55ef755f31', 'status': {'state': 'ready'}}, 'metadata': {'created_at': '2023-11-08T17:07:23.026Z', 'id': '8581afca-3e73-49c8-ac0b-b694ddcdec87', 'modified_at': '2023-11-08T17:07:23.026Z', 'name': 'Batch dep API sleep', 'owner': '1000330999', 'space_id': '7b648ffc-53e3-4b16-8e59-3c55ef755f31'}}]},{'resources': [{'entity': {'asset': {'id': 'b8b849e5-1772-4742-880e-7425cd854838'}, 'batch': {}, 'custom': {}, 'deployed_asset_type': 'function', 'hardware_spec': {'id': 'c076e82c-b2a7-4d20-9c0f-1f0c2fdf5a24', 'name': 'M', 'num_nodes': 1}, 'name': 'Batch dep API sleep', 'space_id': '7b648ffc-53e3-4b16-8e59-3c55ef755f31', 'status': {'state': 'ready'}}, 'metadata': {'created_at': '2023-11-08T17:07:23.026Z', 'id': '8581afca-3e73-49c8-ac0b-b694ddcdec87', 'modified_at': '2023-11-08T17:07:23.026Z', 'name': 'Batch dep API sleep', 'owner': '1000330999', 'space_id': '7b648ffc-53e3-4b16-8e59-3c55ef755f31'}}]}]

             
rdd = sc.parallelize([test_json], 3)
df = spark.read.json(rdd)
df.show()

+--------------------+
|           resources|
+--------------------+
|[{{{b8b849e5-1772...|
|[{{{b8b849e5-1772...|
|[{{{b8b849e5-1772...|
+--------------------+



In [16]:
file_list = ['wx_prompt_details 2.json', 'wx_prompt_details 3.json']#, 'wx_prompt_details 3.json']
#file_list = ['/project_data/data_asset/wx_prompt_details 2.json', '/project_data/data_asset/wx_prompt_details 3.json']


rdd=sc.parallelize(file_list, 2)
rdd = rdd.mapPartitions(process_file)
df5 = spark.read.json(rdd, multiLine=True, encoding="utf8", )
#df5 = spark.read.json(sc.parallelize(file_list, 2), multiLine=True)

'''df5 = spark.read.option("multiline","true") \
          .option("inferSchema", "true") \
          .json(rdd)'''


df5.show()

+--------------------+
|     _corrupt_record|
+--------------------+
|{'metadata': {'sp...|
|{'metadata': {'sp...|
+--------------------+



In [17]:
file_list = ['/project_data/data_asset/wx_prompt_details 2.json', '/project_data/data_asset/wx_prompt_details 3.json']

df4 = spark.read.option("multiline","true") \
          .option("mode", "PERMISSIVE") \
          .option("columnNameOfCorruptRecord", "_corrupt_record") \
          .json(file_list)

In [18]:
df4.columns

['entity', 'metadata']

In [19]:
df4.show()

+--------------------+--------------------+
|              entity|            metadata|
+--------------------+--------------------+
|{{DAL, 1699642358...|{[wx_prompt, loca...|
|{{DAL, 1699642358...|{[wx_prompt, loca...|
+--------------------+--------------------+



### get the json from IBM cos

In [20]:
file_list = ['wx_prompt_details2.json', 'wx_prompt_details3.json']

In [6]:
import ibm_boto3
from ibm_botocore.client import Config, ClientError

# Constants for IBM COS values
COS_ENDPOINT = "https://s3.us-east.cloud-object-storage.appdomain.cloud" # Current list avaiable at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints
COS_API_KEY_ID = "lh6naU4ptbQ-85U6suVxZqXRDT1tdYIBeu4I1aZk1gqn" # eg "W00YixxxxxxxxxxMB-odB-2ySfTrFBIQQWanc--P3byk"
COS_INSTANCE_CRN = "crn:v1:bluemix:public:cloud-object-storage:global:a/0a752745d5084ac9b390f561ecdd729d:3910469f-f46c-426c-b842-e8e5ddbf216f::" # eg "crn:v1:bluemix:public:cloud-object-storage:global:a/3bf0d9003xxxxxxxxxx1c3e97696b71c:d6f04d83-6c4f-4a62-a165-696756d63903::"

# Create client 
cos = ibm_boto3.client("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_INSTANCE_CRN,
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

In [34]:
import ibm_boto3
from ibm_botocore.client import Config, ClientError

# Constants for IBM COS values
COS_ENDPOINT = "https://s3.us-east.cloud-object-storage.appdomain.cloud" # Current list avaiable at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints
COS_API_KEY_ID = "lh6naU4ptbQ-85U6suVxZqXRDT1tdYIBeu4I1aZk1gqn" # eg "W00YixxxxxxxxxxMB-odB-2ySfTrFBIQQWanc--P3byk"
COS_INSTANCE_CRN = "crn:v1:bluemix:public:cloud-object-storage:global:a/0a752745d5084ac9b390f561ecdd729d:3910469f-f46c-426c-b842-e8e5ddbf216f::" # eg "crn:v1:bluemix:public:cloud-object-storage:global:a/3bf0d9003xxxxxxxxxx1c3e97696b71c:d6f04d83-6c4f-4a62-a165-696756d63903::"

# Create resource
cos = ibm_boto3.resource("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_INSTANCE_CRN,
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

In [35]:
def get_buckets():
    print("Retrieving list of buckets")
    try:
        buckets = cos.buckets.all()
        for bucket in buckets:
            print("Bucket Name: {0}".format(bucket.name))
    except ClientError as be:
        print("CLIENT ERROR: {0}\n".format(be))
    except Exception as e:
        print("Unable to retrieve list buckets: {0}".format(e))

In [34]:
get_buckets()

Retrieving list of buckets
Bucket Name: 0daf3acc-ca79-48df-a870-aa20e30c21e8
Bucket Name: assetattributes-donotdelete-pr-iuz0c5jpedaens
Bucket Name: bankingdemoproject-donotdelete-pr-fgmjteouvegwtp
Bucket Name: catfeb21-datacatalog-bdte4zf6cv1ixfx8k3jl
Bucket Name: datafabricoutcomes-donotdelete-pr-zmbbbkvhaxxn6f
Bucket Name: datagovernanceandprivacy-donotdelete-pr-y8jvznt32v8sbf
Bucket Name: datascienceandmlops-donotdelete-pr-oou5btsl2sv0dw
Bucket Name: mlopsandtrustworthyai-donotdelete-pr-uohqgct9g48fyu
Bucket Name: platformassetscatalo-datacatalog-llmwwqgswo09vsnc8ys58
Bucket Name: sql-c8fdbec2-cd80-424f-8882-27e1cbbda3bb
Bucket Name: wml-cos
Bucket Name: wmlpipeline-donotdelete-pr-a17ieegzsjrjnl
Bucket Name: wsfeb17-donotdelete-pr-irdng5mxdojojc


In [21]:
bucket_name='wml-cos'
item_name='wx_prompt_details 2.json'

In [39]:
get_item(bucket_name, item_name)

Retrieving item from bucket: wml-cos, key: wx_prompt_details 2.json
b'{\n  "metadata": {\n    "space_id": "54f757f2-8614-41c4-b875-a8db00ffdff8",\n    "usage": {\n      "last_updated_at": "2023-12-01T18:39:32Z",\n      "last_updater_id": "iam-ServiceId-4e678d10-eb31-4193-9ec3-86c0313ab18d",\n      "last_update_time": 1701455972916,\n      "last_accessed_at": "2023-12-01T18:39:32Z",\n      "last_access_time": 1701455972916,\n      "last_accessor_id": "iam-ServiceId-4e678d10-eb31-4193-9ec3-86c0313ab18d",\n      "access_count": 0\n    },\n    "rov": {\n      "mode": 0,\n      "collaborator_ids": {},\n      "member_roles": {\n        "IBMid-0600019M7F": {\n          "user_iam_id": "IBMid-0600019M7F",\n          "roles": [\n            "OWNER"\n          ]\n        }\n      }\n    },\n    "name": "Insurance claim summarization",\n    "description": "",\n    "asset_type": "wx_prompt",\n    "origin_country": "us",\n    "resource_key": "e1f9002dbd3da61bfd4af64d0df20b93a15a1238f3c11507c3c805900

In [22]:
def get_files(file_list):
    def get_file_content(item_list):
        bucket_name='wml-cos'
        for item in item_list:
            try:
                file = cos.Object(bucket_name, item).get()
                return file["Body"].read().decode("utf-8")
            except ClientError as be:
                return("CLIENT ERROR: {0}\n".format(be))
            except Exception as e:
                return("Unable to retrieve file contents: {0}".format(e))
    return get_file_content(file_list)

In [49]:
file_list = ['wx_prompt_details3.json']
get_files(file_list)

'{\n  "metadata": {\n    "space_id": "54f757f2-8614-41c4-b875-a8db00ffdff8",\n    "usage": {\n      "last_updated_at": "2023-12-01T18:39:32Z",\n      "last_updater_id": "iam-ServiceId-4e678d10-eb31-4193-9ec3-86c0313ab18d",\n      "last_update_time": 1701455972916,\n      "last_accessed_at": "2023-12-01T18:39:32Z",\n      "last_access_time": 1701455972916,\n      "last_accessor_id": "iam-ServiceId-4e678d10-eb31-4193-9ec3-86c0313ab18d",\n      "access_count": 0\n    },\n    "rov": {\n      "mode": 0,\n      "collaborator_ids": {},\n      "member_roles": {\n        "IBMid-0600019M7F": {\n          "user_iam_id": "IBMid-0600019M7F",\n          "roles": [\n            "OWNER"\n          ]\n        }\n      }\n    },\n    "name": "Insurance claim summarization",\n    "description": "",\n    "asset_type": "wx_prompt",\n    "origin_country": "us",\n    "resource_key": "e1f9002dbd3da61bfd4af64d0df20b93a15a1238f3c11507c3c8059001ed310c",\n    "rating": 0,\n    "total_ratings": 0,\n    "catalog_id

In [35]:
base_url = None
base_public_url = None
def readFilesFromCOS():
    bucket = 'wml-cos'
    api_key_id = 'lh6naU4ptbQ-85U6suVxZqXRDT1tdYIBeu4I1aZk1gqn'
    service_instance_id = 'crn:v1:bluemix:public:cloud-object-storage:global:a/0a752745d5084ac9b390f561ecdd729d:3910469f-f46c-426c-b842-e8e5ddbf216f::'
    endpoint_url = 'https://s3.us-east.cloud-object-storage.appdomain.cloud'
    
    
    # listing all the files in the bucket
    from ibm_botocore.client import Config
    import ibm_boto3
    cos_client = ibm_boto3.client(service_name='s3',
        ibm_api_key_id=api_key_id,
        ibm_service_instance_id=service_instance_id,
        #ibm_auth_endpoint='https://s3.ca-tor.cloud-object-storage.appdomain.cloud',
        config=Config(signature_version='oauth'),
        endpoint_url=endpoint_url)
    
    paginator = cos_client.get_paginator('list_objects_v2') # using paginator as this will allow reading more than 1000 files if required. 

    pages = paginator.paginate(Bucket=bucket)

    file_list = []
    for page in pages:
        for obj in page['Contents']:
            file_list.append(obj['Key'])
    json_files = [x for x in file_list if x.endswith('.json')] #partitionDate
    
    #[print(x) for x in file_list if x.startswith('20230828/STR/') and x.endswith('.json')]
    
    config_name = 'service'
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(f"fs.cos.{config_name}.endpoint", endpoint_url)
    hconf.set(f"fs.cos.{config_name}.iam.service.id",service_instance_id)
    hconf.set(f"fs.cos.{config_name}.iam.api.key",api_key_id)

    public_endpoint = 's3.ca-tor.cloud-object-storage.appdomain.cloud'
    print("public endpoint:" + public_endpoint)
    
    global base_url 
    global base_public_url 
    
    base_url = 'cos://' + bucket + '.' + config_name + '/'
    base_public_url = 'https://' + public_endpoint + '/' + bucket + '/'
    print("base public url:" + base_public_url)

    cos_json_files = [base_url + x for x in json_files]
    # selecting only one file which is the sample STR for the MVP : change agreed on Aug 15 with IBM + Versatil
    #cos_json_files = 'cos://dirty-data.service/20230725/STR_db_tables_excample_data_v09.json'
    print("Loaded.")
    return cos_json_files

In [24]:
readFilesFromCOS()

public endpoint:s3.ca-tor.cloud-object-storage.appdomain.cloud
base public url:https://s3.ca-tor.cloud-object-storage.appdomain.cloud/wml-cos/
Loaded.


['cos://wml-cos.service/wx_prompt_details2.json',
 'cos://wml-cos.service/wx_prompt_details3.json']

In [25]:
file_list = readFilesFromCOS()

df5 = spark.read.option("multiline","true") \
            .option("inferSchema", "true")\
          .json(file_list)


df5.show()

public endpoint:s3.ca-tor.cloud-object-storage.appdomain.cloud
base public url:https://s3.ca-tor.cloud-object-storage.appdomain.cloud/wml-cos/
Loaded.
+--------------------+--------------------+
|              entity|            metadata|
+--------------------+--------------------+
|{{DAL, 1699642358...|{[wx_prompt, loca...|
|{{DAL, 1699642358...|{[wx_prompt, loca...|
+--------------------+--------------------+



In [37]:
file_list = readFilesFromCOS()

df5 = spark.read.option("multiline","true") \
            .option("inferSchema", "true")\
            .option("numPartitions",4)\
          .json(file_list)


df5.show()

public endpoint:s3.ca-tor.cloud-object-storage.appdomain.cloud
base public url:https://s3.ca-tor.cloud-object-storage.appdomain.cloud/wml-cos/
Loaded.
+--------------------+--------------------+
|              entity|            metadata|
+--------------------+--------------------+
|{{DAL, 1699642358...|{[wx_prompt, loca...|
|{{DAL, 1699642358...|{[wx_prompt, loca...|
+--------------------+--------------------+



In [ ]:
    '''PaginationConfig={'MaxItems': 2,
                    'PageSize': 2,
                    })'''
    
    PaginationConfig={'PageSize': 10})

In [19]:
api_key_id = 'lh6naU4ptbQ-85U6suVxZqXRDT1tdYIBeu4I1aZk1gqn'
service_instance_id = 'crn:v1:bluemix:public:cloud-object-storage:global:a/0a752745d5084ac9b390f561ecdd729d:3910469f-f46c-426c-b842-e8e5ddbf216f::'
endpoint_url = 'https://s3.us-east.cloud-object-storage.appdomain.cloud'
bucket_name='wml-cos'

def get_file_list():
    from ibm_botocore.client import Config
    import ibm_boto3
    cos_client = ibm_boto3.client(service_name='s3',
        ibm_api_key_id=api_key_id,
        ibm_service_instance_id=service_instance_id,
        #ibm_auth_endpoint='https://s3.ca-tor.cloud-object-storage.appdomain.cloud',
        config=Config(signature_version='oauth'),
        endpoint_url=endpoint_url)
    
    
    paginator = cos_client.get_paginator('list_objects_v2') # using paginator as this will allow reading more than 1000 files if required. 

    pages = paginator.paginate(Bucket=bucket_name,PaginationConfig={'MaxItems': 10,'PageSize': 3, 'Prefix': 'STR_Files/STR-'})


    file_list = []
    for page in pages:
        for obj in page['Contents']:
            file_list.append(obj['Key'])
    json_files = [x for x in file_list if x.endswith('.json')]
    return json_files


def get_files(file_list):

    
    # listing all the files in the bucket
    from ibm_botocore.client import Config
    import ibm_boto3
    cos_res = ibm_boto3.resource(service_name='s3',
        ibm_api_key_id=api_key_id,
        ibm_service_instance_id=service_instance_id,
        #ibm_auth_endpoint='https://s3.ca-tor.cloud-object-storage.appdomain.cloud',
        config=Config(signature_version='oauth'),
        endpoint_url=endpoint_url)



    def get_file_content(cos_res, item_list):
        
        for item in item_list:
            try:
                file = cos_res.Object(bucket_name, item).get()
                yield file["Body"].read().decode("utf-8")
            except ClientError as be:
                return("CLIENT ERROR: {0}\n".format(be))
            except Exception as e:
                return("Unable to retrieve file contents: {0}".format(e))
    return get_file_content(cos_res, file_list)

In [20]:
#file_list = ['wx_prompt_details3.json', 'wx_prompt_details2.json']
file_list = get_file_list()

rdd= sc.parallelize(file_list, 1)
rdd = rdd.mapPartitions(get_files)
df5 = spark.read.json(rdd, multiLine=True, encoding="utf8")
#df5 = spark.read.json(sc.parallelize(file_list, 2), multiLine=True)


df5.show()

+--------------------+
|     _corrupt_record|
+--------------------+
|{\n  "_disclaimer...|
|{\n  "_disclaimer...|
|{\n  "_disclaimer...|
|{\n  "_disclaimer...|
|{\n  "_disclaimer...|
|{\n  "_disclaimer...|
|{\n  "_disclaimer...|
|{\n  "_disclaimer...|
|{\n  "_disclaimer...|
|{\n  "_disclaimer...|
+--------------------+



In [42]:
#file_list = ['wx_prompt_details3.json', 'wx_prompt_details2.json']
file_list = get_file_list()
rdd= sc.parallelize(file_list, 2)
rdd = rdd.mapPartitions(get_files)
df5 = spark.read.json(rdd, multiLine=True, encoding="utf8")

df5.show()

+--------------------+--------------------+
|              entity|            metadata|
+--------------------+--------------------+
|{{DAL, 1699642358...|{[wx_prompt, loca...|
|{{DAL, 1699642358...|{[wx_prompt, loca...|
+--------------------+--------------------+



In [18]:
#file_list = ['wx_prompt_details3.json', 'wx_prompt_details2.json']
file_list = get_file_list()

rdd= sc.parallelize(file_list, 2)
rdd = rdd.mapPartitions(get_files)
df5 = spark.read.json(rdd, multiLine=True, encoding="utf8")
#df5 = spark.read.json(sc.parallelize(file_list, 2), multiLine=True)


df5.show()

+--------------------+--------------------+
|              entity|            metadata|
+--------------------+--------------------+
|{{DAL, 1699642358...|{[wx_prompt, loca...|
|{{DAL, 1699642358...|{[wx_prompt, loca...|
+--------------------+--------------------+

